<a href="https://colab.research.google.com/github/wolfsbane9513/Android-Lib-VerticalMarqueeTextView/blob/master/Distributed_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stage 1: Installing dependencies and setting up the environment

In [1]:
!pip install tensorflow-gpu==2.0.0.0

     |████████████████████████████████| 380.8MB 48kB/s 
     |████████████████████████████████| 450kB 44.8MB/s 
     |████████████████████████████████| 3.8MB 33.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=60473c48b9fbce7dbbb667f5cd58ec7a43dd57038d4d328da55a136e5fb1036e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc3 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0rc0


## Stage 2: Importing project dependencies

In [0]:
import time
import numpy as np
import tensorflow as tf

In [3]:
tf.__version__

'2.0.0'

## Stage 3: Dataset preprocessing

### Loading the MNIST dataset

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Image normalization

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

In [0]:
X_train.shape

(60000, 28, 28)

### Dataset reshaping

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [8]:
X_train.shape

(60000, 784)

## Stage 4: Distributed Training

### Defining normal (non distributed) model

In [0]:
model_normal = tf.keras.models.Sequential()

In [0]:
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))

In [0]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))

In [0]:
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [0]:
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Defining the Distributed Strategy

In [0]:
distribute = tf.distribute.MirroredStrategy()

### Defining a distributed model

In [15]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


### Speed comparison between normal training and distributed training process

In [16]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("Distributed training took: {}".format(time.time() - start_time))

Train on 60000 samples
Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
60000/60000 [==============================] - 12s 206us/sample - loss: 1.1566 - sparse_categorical_accuracy: 0.7016
Epoch 2/10
60000/60000 [==============================] - 8s 139us/sample - loss: 0.5190 - sparse_categorical_accuracy: 0.8552
Epoch 3/10
60000/60000 [==============================] - 8s 139us/sample - loss: 0.4140 - sparse_categorical_accuracy: 0.8819
Epoch 4/10
60000/60000 [===========

In [17]:
start_time = time.time()
model_normal.fit(X_train, y_train, epochs=10, batch_size=25)
print("Normal training took: {}".format(time.time() - start_time))

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 135us/sample - loss: 1.1140 - sparse_categorical_accuracy: 0.7111
Epoch 2/10
60000/60000 [==============================] - 8s 130us/sample - loss: 0.4917 - sparse_categorical_accuracy: 0.8645
Epoch 3/10
60000/60000 [==============================] - 8s 131us/sample - loss: 0.3966 - sparse_categorical_accuracy: 0.8871
Epoch 4/10
60000/60000 [==============================] - 8s 130us/sample - loss: 0.3518 - sparse_categorical_accuracy: 0.8978
Epoch 5/10
60000/60000 [==============================] - 8s 129us/sample - loss: 0.3263 - sparse_categorical_accuracy: 0.9060
Epoch 6/10
60000/60000 [==============================] - 8s 129us/sample - loss: 0.3024 - sparse_categorical_accuracy: 0.9123
Epoch 7/10
60000/60000 [==============================] - 8s 131us/sample - loss: 0.2859 - sparse_categorical_accuracy: 0.9167
Epoch 8/10
60000/60000 [==============================] - 8s 130us/sample - loss: 0.2686